This file runs model inference for a 3B parameter model.

In [ ]:
!pip install transformers
!pip install accelerate
!pip install google-colab
!pip install -q -U bitsandbytes
!pip install torch torch_xla
!pip install einops
#https://huggingface.co/togethercomputer/RedPajama-INCITE-Instruct-3B-v1?text=1.+The+food+was+great+quality.+Their+tofu+was+very+great.+The+size+of+each+dish+is+pretty+big+so+you%27ll+leave+with+a+full+stomach.+The+waiting+time+was+a+few+minutes%2C+I+was+impressed+by+how+fast+the+service+was.%0D%0A%0D%0A2.+Just+had+the+Pad+Thai%2C+hands+down+the+worst+Pad+Thai+I+have+ever+had.+I+have+had+their+Pad+Thai+in+their+other+locations+and+it+was+very+good.+Not+sure+what+happened+here+tonight%2C+but+it+was+a+real+bummer.+Table+Service+was+great%2C+prompt+and+fast.%0D%0A%0D%0A3.+Just+a+short+walking+distance+from+the+UT+campus.+The+restaurant+has+indoor+and+outdoor+seating.+A+bit+pricy+but+the+quality+of+the+food+really+compensates+for+it.%0D%0A%0D%0APlease+provide+a+detailed%2C+objective+summary+for+each+of+the+following+categories+based+on+the+provided+restaurant+reviews%3A+Ambiance%2C+Food%2C+and+Service.+Ensure+each+summary+focuses+on+different+aspects+as+mentioned+in+the+reviews%2C+avoiding+repetition+and+first-person+narratives.+Please+do+not+include+numerical+ratings%2C+but+rather+focus+on+qualitative+descriptions+that+clearly+convey+the+reviewers%27+experiences+and+opinions+in+each+category.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-3B-v1", torch_dtype=torch.float16)
model = model.to('cuda:0')

In [ ]:
prompt = """
1. The food was great quality. Their tofu was very great. The size of each dish is pretty big so you'll leave with a full stomach. The waiting time was a few minutes, I was impressed by how fast the service was.

2. Just had the Pad Thai, hands down the worst Pad Thai I have ever had. I have had their Pad Thai in their other locations and it was very good. Not sure what happened here tonight, but it was a real bummer. Table Service was great, prompt and fast.

3. Just a short walking distance from the UT campus. The restaurant has indoor and outdoor seating. A bit pricy but the quality of the food really compensates for it.

4. Been searching Austin for a while now to find the best Thai food, and was recommended Madam Man's by a friend. Unfortunately I still have not found the best Thai yet. We ordered to-go and it was ready for pickup upon arrival. It looked like a lot of orders are pickup vs dine in (but this was a Sunday night). The restaurant looked like it had a nice patio but a bit of a sparse interior. For food we ordered Pad See Ew, pad Thai and Thai tea. Didn't love any of these, but I have heard their Thai chicken wings are very good so I may try again. The pad Thai was overly "sauce" flavored but yet somehow still dry. Tasted more like lime than peanut or anything I'm used to. The Thai tea was very watery (but it had been sitting for a while waiting on us). The pad see ew was the best dish we had and was ordered as a reviewer had recommended (chicken, extra sauce, spicy 3). First of all, if their spicy scale was supposed to be out of 10 (I asked when ordering), that was a pretty darn spicy 3. The noodles did not seem like they had extra sauce (they seemed to have very little sauce at all) - overall just not what I was hoping for. I love places on Campus like this and want to try some of their other things, but first time wasn't the charm.

Please provide a objective summary for the restaurant based on the provided reviews. Ensure the summary focuses on different aspects as mentioned in the reviews, avoiding repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions.

"""
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    **inputs, max_new_tokens=128, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True
)
token = outputs.sequences[0, input_length:]
output_str = tokenizer.decode(token)
print(output_str)